## **Contar tokens**
Este notebook tiene como objetivo contar el número de tokens en el prompt usado por el chatbot, ya que algunos modelos como `GPT-2` tienen un límite de tokens (1024). 

---
### **Librerías**

In [5]:
from transformers import LlamaTokenizer
import tiktoken

/opt/miniconda3/envs/PharmAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### **1. Definir textos**

In [6]:
# Prompt para el modelo Llama 2 Chat 7B
prompt_llama2 = """
    1. OBJETIVO GENERAL:
    Eres un asistente médico especializado en información sobre medicamentos. Debes responder a la pregunta del usuario basándote únicamente en la información proporcionada. No debes inventar ni suponer información adicional.

    2. FORMATO DEL CONTEXTO:
    El contexto se presenta como un texto con varios fragmentos que contiene información de uno o varios medicamentos presentes en la pregunta del ususario, donde cada fragmento tiene el siguiente formato:
    - Medicamento: Nombre del medicamento
    - Categoría: Categoría de la información (ej. efectos secundarios, interacciones)
    - Información: Texto relevante sobre el medicamento, el cual debes analizar antes de responder.

    3. FORMATO DE RESPUESTA:
    - Debes responder de manera clara y precisa a la pregunta formulada por el usuario, utilizando ÚNICAMENTE el contexto que se te está proporcionando. 
    - No debes inventar información ni suponer datos que no estén presentes en el contexto.
    - Si la información proporcionada no es suficiente para responder completamente, dilo e indica qué datos faltan.
    - Incluye una mención explícita a los textos que respaldan tu respuesta, indicando para todos ellos el medicamento y la categoría.
    - Si la pregunta no está relacionada con medicamentos, indica que no puedes ayudar en ese caso.

    4. EJEMPLO DE CONSULTA Y DE RESPUESTA ESPERADA:
    Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?
    Contexto:
        "medicamento": "Aspirina"
        "categoria": "efectos_secundarios"
        "texto": "Puede causar náuseas y dolor de estómago."
    Respuesta:
    La aspirina puede causar efectos secundarios como náuseas y dolor de estómago (extraído de la ficha técnica, de la sección "efectos_secundarios" del medicamento "ASPIRINA": "la aspirina tiene como efectos secundarios, entre ottros, la aparición de náuseas y dolor de tripa"). Si necesitas más detalles, por favor consulta la ficha técnica completa.

    5. INSTRUCCIONES FINALES:
    Básandote ÚNICAMENTE en la información proporcionada en ({context}), responde a la siguiente pregunta:
    {query}
    Respuesta:"""

In [7]:
# Prompt para el modelo GPT-2
prompt_gpt2 = """
Eres un asistente médico especializado en información sobre medicamentos. Debes responder a la pregunta del usuario basándote únicamente en la información proporcionada. No debes inventar ni suponer información adicional. Si recibes un contexto que no aporta información sobre medicamentos, responde que no puedes responder a la pregunta.

Contexto:
{context}

Pregunta: {query}
Respuesta:"""

In [8]:
# Contexto con 10 fragmentos
contexto_10_fragmentos = """Contexto: 
Fragmento 1:
Medicamento: ibuprofeno sandoz
Categoría: efectos_conducir
Información: para el uso a corto plazo de ibuprofeno, no es necesario tomar precauciones especiales.

Fragmento 2:
Medicamento: ibuprofeno sandoz
Categoría: efectos_conducir
Información: para el uso a corto plazo de ibuprofeno, no es necesario tomar precauciones especiales.

Fragmento 3:
Medicamento: ibuprofeno ambah
Categoría: incompatibilidades
Información: ibuprofeno ambah no debe mezclarse con otros medicamentos.

Fragmento 4:
Medicamento: ibuprofeno cinfa
Categoría: indicaciones
Información: ibuprofeno está indicado en niños a partir de 3 meses y hasta 12 años para el: alivio sintomático de los dolores ocasionales leves o moderados. estados febriles.

Fragmento 5:
Medicamento: eudorlin infantil
Categoría: interacciones
Información: el ibuprofeno (al igual que otros aine) debe combinarse con precaución con los siguientes medicamentos: otros aine, incluyendo salicilatos: la administración concomitante de diferentes aine puede aumentar el riesgo de úlceras gastrointestinales y hemorragias debido a un efecto sinérgico. por tanto, debe evitarse el uso concomitante de ibuprofeno con otros aine digoxina, fenitoina, litio: el uso concomitante de eudorlin infantil con digoxina, fenitoina y litio puede aumentar los niveles séricos de estos medicamentos. generalmente, no se requiere un control de los niveles séricos de digoxina, fenitoina y litio si se usan correctamente (máximo a partir de los 3 días). diuréticos, inhibidores del eca, betabloqueantes y antagonistas de angiotensina ii: los aine pueden reducir el efecto de los diuréticos y de otros medicamentos antihipertensivos. en pacientes con deterioro de la función renal (p. ej., pacientes deshidratados o pacientes de edad avanzada con deterioro de la función renal) la administración combinada de inhibidores de eca, betabloqueantes o antagonistas de la angiotensina ii y agentes que inhiben la ciclooxigenasa pueden ocasionar un deterioro adicional de la función renal, incluso una posible insuficiencia renal aguda, que normalmente es reversible. por tanto, esta combinación debe administrarse con precaución, especialmente en personas de edad avanzada. los pacientes deberán hidratarse adecuadamente y una vez comenzado el tratamiento concomitante deberán someterse a un control periódico de la función renal. la administración concomitante de eudorlin infantil y diuréticos ahorradores de potasio puede dar lugar a hiperpotasemia. corticosteroides: aumento del riesgo de úlcera o hemorragia gastrointestinal agentes antiagragantes plaquetarios e inhibidores selectivos de la recaptación de serotonina (isrs): aumento del riesgo de hemorragia gastrointestinal ácido acetilsalicílico en general, no se recomienda la administración concomitante de ibuprofeno y ácido a...

Fragmento 6:
Medicamento: neobrufen con codeina
Categoría: advertencias
Información: las reacciones adversas se pueden minimizar utilizando la dosis eficaz más baja durante el menor tiempo necesario para aliviar los síntomas y secciones posteriores). al igual que sucede con otros opioides, si el control del dolor es insuficiente pese al aumento de la dosis de codeína, debe considerarse la posibilidad de hiperalgesia inducida por fármacos. puede estar indicada la reducción de la dosis o la revisión del tratamiento. advertencias debidas al ibuprofeno enmascaramiento de los síntomas de infecciones subyacentes: como cualquier otro aine, el ibuprofeno puede enmascarar los síntomas de una infección, lo que puede retrasar el inicio del tratamiento adecuado y, por tanto, empeorar el desenlace de la infección. esto se ha observado en la neumonía bacteriana extrahospitalaria y en las complicaciones bacterianas de la varicela. cuando se administre ibuprofeno para aliviar la fiebre o el dolor relacionados con una infección, se recomienda vigilar la infección. en entornos no hospitalarios, el paciente debe consultar a un médico si los síntomas persisten o empeoran. con el uso prolongado de analgésicos pueden aparecer dolores de cabeza que no deben ser tratados incrementando la dosis del producto. las reacciones adversas provocadas por ibuprofeno, especialmente las relacionadas con el tracto gastrointestinal o sistema nervioso, pueden verse aumentadas por el consumo de alcohol. riesgos gastrointestinales hemorragias gastrointestinales, úlceras y perforaciones: durante el tratamiento con antiinflamatorios no esteroideos (aines) entre ibuprofeno, se han notificado hemorragias gastrointestinales, úlceras y perforaciones (que pueden ser mortales) en cualquier momento del mismo, con o sin síntomas previos de alerta y con o sin antecedentes previos de acontecimientos gastrointestinales graves previos los que se encuentra el riesgo de hemorragia gastrointestinal, úlcera o perforación es mayor cuando se utilizan dosis crecientes de aines, en pacientes con antecedentes de...

Fragmento 7:
Medicamento: ibuprofeno aristogen
Categoría: efectos_conducir
Información: el ibuprofeno generalmente tiene una influencia nula o insignificante en la capacidad para conducir y utilizar máquinas. sin embargo, dado que a dosis más altas pueden producirse efectos adversos en el sistema nervioso central, como cansancio y mareos, los pacientes que están en tratamiento con ibuprofeno pueden ver afectado su tiempo de reacción, lo que debe ser tenido en cuenta a la hora de realizar actividades que requieren una mayor vigilancia, como por ejemplo conducir o utilizar maquinaria. esto se aplica en mayor medida a la combinación con alcohol.

Fragmento 8:
Medicamento: difenadol
Categoría: efectos_conducir
Información: ibuprofeno generalmente tiene una influencia nula o insignificante en la capacidad para conducir y utilizar máquinas. sin embargo, dado que a dosis más altas pueden producirse efectos adversos en el sistema nervioso central, como cansancio y mareos, los pacientes que están en tratamiento con ibuprofeno pueden ver afectado su tiempo de reacción, lo que debe ser tenido en cuenta a la hora de realizar actividades que requieren una mayor vigilancia, como por ejemplo conducir o utilizar maquinaria. esto se aplica en mayor medida a la combinación con alcohol.

Fragmento 9:
Medicamento: diltix
Categoría: efectos_conducir
Información: ibuprofeno generalmente tiene una influencia nula o insignificante en la capacidad para conducir y utilizar máquinas. sin embargo, dado que a dosis más altas pueden producirse efectos adversos en el sistema nervioso central, como cansancio y mareos, los pacientes que están en tratamiento con ibuprofeno pueden ver afectado su tiempo de reacción, lo que debe ser tenido en cuenta a la hora de realizar actividades que requieren una mayor vigilancia, como por ejemplo conducir o utilizar maquinaria. esto se aplica en mayor medida a la combinación con alcohol.

Fragmento 10:
Medicamento: diltix
Categoría: efectos_conducir
Información: ibuprofeno generalmente tiene una influencia nula o insignificante en la capacidad para conducir y utilizar máquinas. sin embargo, dado que a dosis más altas pueden producirse efectos adversos en el sistema nervioso central, como cansancio y mareos, los pacientes que están en tratamiento con ibuprofeno pueden ver afectado su tiempo de reacción, lo que debe ser tenido en cuenta a la hora de realizar actividades que requieren una mayor vigilancia, como por ejemplo conducir o utilizar maquinaria. esto se aplica en mayor medida a la combinación con alcohol.


"""

In [19]:
# Contexto con 3 fragmentos
contexto_3_fragmentos = """Contexto: 
Fragmento 1:
Medicamento: ibuprofeno sandoz
Categoría: efectos_conducir
Información: para el uso a corto plazo de ibuprofeno, no es necesario tomar precauciones especiales.

Fragmento 2:
Medicamento: ibuprofeno sandoz
Categoría: efectos_conducir
Información: para el uso a corto plazo de ibuprofeno, no es necesario tomar precauciones especiales.

Fragmento 3:
Medicamento: eudorlin infantil
Categoría: interacciones
Información: el ibuprofeno (al igual que otros aine) debe combinarse con precaución con los siguientes medicamentos: otros aine, incluyendo salicilatos: la administración concomitante de diferentes aine puede aumentar el riesgo de úlceras gastrointestinales y hemorragias debido a un efecto sinérgico. por tanto, debe evitarse el uso concomitante de ibuprofeno con otros aine digoxina, fenitoina, litio: el uso concomitante de eudorlin infantil con digoxina, fenitoina y litio puede aumentar los niveles séricos de estos medicamentos. generalmente, no se requiere un control de los niveles séricos de digoxina, fenitoina y litio si se usan correctamente (máximo a partir de los 3 días). diuréticos, inhibidores del eca, betabloqueantes y antagonistas de angiotensina ii: los aine pueden reducir el efecto de los diuréticos y de otros medicamentos antihipertensivos. en pacientes con deterioro de la función renal (p. ej., pacientes deshidratados o pacientes de edad avanzada con deterioro de la función renal) la administración combinada de inhibidores de eca, betabloqueantes o antagonistas de la angiotensina ii y agentes que inhiben la ciclooxigenasa pueden ocasionar un deterioro adicional de la función renal, incluso una posible insuficiencia renal aguda, que normalmente es reversible. por tanto, esta combinación debe administrarse con precaución, especialmente en personas de edad avanzada. los pacientes deberán hidratarse adecuadamente y una vez comenzado el tratamiento concomitante deberán someterse a un control periódico de la función renal. la administración concomitante de eudorlin infantil y diuréticos ahorradores de potasio puede dar lugar a hiperpotasemia. corticosteroides: aumento del riesgo de úlcera o hemorragia gastrointestinal agentes antiagragantes plaquetarios e inhibidores selectivos de la recaptación de serotonina (isrs): aumento del riesgo de hemorragia gastrointestinal ácido acetilsalicílico en general, no se recomienda la administración concomitante de ibuprofeno y ácido a...
"""

---
### **2. Contar tokens (modelo `Llama 2 chat 7b`)**

In [10]:
def count_tokens_llama2(text):
    # Carga el tokenizer de Llama 2 Chat 7B
    tokenizer_llama2 = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

    # Obtén los IDs de tokens y cuenta
    ids = tokenizer_llama2(text).input_ids
    num_tokens = len(ids)

    print(f"Número de tokens: {num_tokens}")

    return num_tokens

In [11]:
# Prompt 
count_tokens_llama2(prompt_llama2)

Número de tokens: 657


657

In [12]:
# Contexto con 10 fragmentos
count_tokens_llama2(contexto_10_fragmentos)

Número de tokens: 2200


2200

---
### **3. Contar tokens (modelo `GPT-2`)**

In [20]:
def num_tokens_gpt2(texto: str) -> int:
    """
    Cuenta el número de tokens usando el tokenizador GPT-2 de tiktoken.
    """
    encoding = tiktoken.get_encoding("gpt2")
    num_tokens = len(encoding.encode(texto))
    print(f"Número de tokens: {num_tokens}")

In [21]:
# Prompt
num_tokens_gpt2(prompt_gpt2)

Número de tokens: 133


In [22]:
# Contexto 3 fragmentos
num_tokens_gpt2(contexto_3_fragmentos)

Número de tokens: 843


Con menos fragmentos (3 vs 10) y un prompt mucho más escueto, conseguimos reducir el número de tokens